# PDF NER

> Functions for extracting text from PDFs and running NER on them.

In [ ]:
# | default_exp pdf.ner

In [ ]:
# | export

from dreamai_ray.imports import *

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# | export


def load_ner_model(model_name="tner/deberta-v3-large-ontonotes5", device="cpu"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    return pipeline(
        "ner",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="simple",
        device=device,
    )


def load_job_model(model_name="ismail-lucifer011/autotrain-job_all-903929564", device="cpu"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(model_name)
    return pipeline(
        "ner",
        model=model,
        tokenizer=tokenizer,
        aggregation_strategy="simple",
        device=device,
    )


def proc_ner(txt, ner, ner_dict={"institute": "", "date": ""}, thresh=3):
    ner_dict = copy.deepcopy(ner_dict)
    org_key = "institute" if "institute" in ner_dict else "company"
    mapper = {
        "ORG": org_key,
        "FAC": org_key,
        "GPE": org_key,
        "LOC": org_key,
        "Job": "role",
        "WORK_OF_ART": "degree",
        "DATE": "date",
    }
    ner_dict["text"] = txt
    for d in ner:
        eg = d["entity_group"]
        w = " " + d["word"].strip()
        k = mapper.get(eg, None)
        if k is not None and ner_dict.get(k, None) is not None and not w.startswith("##"):
            ner_dict[k] = (ner_dict[k] + w).strip()
    res = {k: v for k, v in ner_dict.items() if len(v) > thresh}
    if res.get(org_key, None) is None:
        return {}
    return res


def job_ner(txt, tner, jner):
    return tner(txt) + jner(txt)


def edu_ner(txt, tner, ner_dict={"institute": "", "date": ""}):
    ner = tner(txt)
    return proc_ner(txt, ner, ner_dict)


def work_ner(txt, tner, jner, ner_dict={"company": "", "date": ""}):
    ner = job_ner(txt, tner, jner)
    return proc_ner(txt, ner, ner_dict)


def is_valid_jner(ner, thresh=3):
    return ner.get("company", None) is not None


def is_valid_tner(ner, thresh=3):
    return ner.get("institute", None) is not None

In [ ]:
# | hide

import nbdev

nbdev.nbdev_export()
